In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install transformers==2.10.0

In [0]:
!pip install simpletransformers

In [0]:
!pip install wandb

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 466ed0b4b2a230df648b8b1636be583c783c283e
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [1]:
cd "/content/drive/My Drive/Colab Notebooks/sentiment"

/content/drive/My Drive/Colab Notebooks/sentiment


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from simpletransformers.classification import ClassificationModel
import logging
import numpy as np
import sklearn
import wandb

In [4]:
def modify_sentence(raw_sentence):
    words = word_tokenize(raw_sentence)
    sentence = ' '.join(words)
    
    return sentence


prefix = 'data/'

train_data = pd.read_excel(prefix + "train_data.xlsx")
train_data.columns = ['text_a', 'text_b', 'labels']
train_data['text_a'] = train_data['text_a'].apply(lambda x: modify_sentence(x))
train_data['text_b'] = train_data['text_b'].apply(lambda x: modify_sentence(x))
train_data["labels"].replace({"positive": "1", "negative": "0"}, inplace=True)
train_data = train_data.astype({"labels": int})
print(train_data.head(5))
print(train_data.groupby('labels').count())


eval_data = pd.read_excel(prefix + "test_data.xlsx")
eval_data.columns = ['text_a', 'text_b', 'labels']
eval_data['text_a'] = eval_data['text_a'].apply(lambda x: modify_sentence(x))
eval_data['text_b'] = eval_data['text_b'].apply(lambda x: modify_sentence(x))
eval_data["labels"].replace({"positive": "1", "negative": "0"}, inplace=True)
eval_data = eval_data.astype({"labels": int})
print(eval_data.head(5))
print(eval_data.groupby('labels').count())

                                              text_a  ... labels
0  Adding household quarantine to isolation of th...  ...      1
1  Adding household quarantine to isolation of th...  ...      1
2  Adding household quarantine to isolation of th...  ...      1
3  Preventing mass gatherings is expected to have...  ...      0
4  Preventing mass gatherings is expected to have...  ...      1

[5 rows x 3 columns]
        text_a  text_b
labels                
0          141     141
1          198     198
                                              text_a  ... labels
0  Stopping big gatherings can have cascading eff...  ...      1
1  Cancellation of gatherings of 250 or more peop...  ...      1
2  Compared to social distancing , home quarantin...  ...      0
3  Compared to social distancing , home quarantin...  ...      1
4  Invoking stopping mass gatherings could come w...  ...      0

[5 rows x 3 columns]
        text_a  text_b
labels                
0           40      40
1           73 

In [0]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args={
    'fp16': False,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    
    'evaluate_during_training': True,
    'evaluate_during_training_steps': 4,
    "evaluate_during_training_verbose": True,
    'logging_steps':4,
    
    'num_train_epochs': 5,
    
    'learning_rate':3.5e-5,
    
    'train_batch_size':16,
    
    'warmup_ratio': 0.1, 
    'warmup_steps':16,

    'weight_decay':0.001,

    'gradient_accumulation_steps':2,
    
    "wandb_project": "NPI_sentiment",
    
    'save_eval_checkpoints': False,
    "save_model_every_epoch": False,
    "use_cached_eval_features": True,
    
    "manual_seed": 1
}



In [6]:
# Create a NERModel
model = ClassificationModel('xlnet', 
                            'xlnet-large-cased',
                            args=train_args,
                            num_labels=2)


# Train the model
model.train_model(train_data,
                  eval_df = eval_data,
                  acc = sklearn.metrics.accuracy_score,
                  f1_score = sklearn.metrics.f1_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjI3dHJyZGE3Ok5QSV9zZW50aW1lbnQ6cnlhbmxlZQ==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/requirements.txt
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-history.jsonl
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-summary.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-events.jsonl


Running loss: 0.934663

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-summary.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/media/graph/graph_0_summary_b9a3516b.graph.json
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/media/graph
INFO:wandb.run_manager:file/dir created: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/media


...

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-history.jsonl
INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_xlnet_128_2_113
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-summary.json
INFO:simpletransformers.classification.classification_model:{'mcc': 0.926972334318979, 'tp': 69, 'tn': 40, 'fp': 0, 'fn': 4, 'acc': 0.9646017699115044, 'f1_score': 0.971830985915493, 'eval_loss': 0.12749964892864227}
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Col

Running loss: 0.017433

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json
INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_xlnet_128_2_113


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9444595400934477, 'tp': 70, 'tn': 40, 'fp': 0, 'fn': 3, 'acc': 0.9734513274336283, 'f1_score': 0.9790209790209791, 'eval_loss': 0.12109338343143464}
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /conte

INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json
INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-events.jsonl
INFO:wandb.run_manager:stopping streaming files and file change observer
INFO:wandb.run_manager:file/dir modified: /content/drive/My Drive/Colab Notebooks/sentiment/wandb/run-20200605_114447-27trrda7/wandb-metadata.json


In [7]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_data, 
                                                            acc = sklearn.metrics.accuracy_score,
                                                            f1_score = sklearn.metrics.f1_score)

print("result:", result)

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_xlnet_128_2_113


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9444595400934477, 'tp': 70, 'tn': 40, 'fp': 0, 'fn': 3, 'acc': 0.9734513274336283, 'f1_score': 0.9790209790209791, 'eval_loss': 0.12109338343143464}



result: {'mcc': 0.9444595400934477, 'tp': 70, 'tn': 40, 'fp': 0, 'fn': 3, 'acc': 0.9734513274336283, 'f1_score': 0.9790209790209791, 'eval_loss': 0.12109338343143464}


In [13]:
predictions, raw_outputs = model.predict([["In China , wearing masks , enhanced testing and public building closures haven’t succeeded in reducing the incidence of cases , compared to social distancing .", "wearing masks"],
                                          ["In China , wearing masks , enhanced testing and public building closures haven’t succeeded in reducing the incidence of cases , compared to social distancing .", "enhanced testing"],
                                          ["In China , wearing masks , enhanced testing and public building closures haven’t succeeded in reducing the incidence of cases , compared to social distancing .", "public building closures"],
                                          ["In China , wearing masks , enhanced testing and public building closures haven’t succeeded in reducing the incidence of cases , compared to social distancing .", "social distancing"],
                                          ['Wearing a mask is the worst way to prevent the virus , but using hand sanitizer is effective .', 'Wearing masks'],
                                          ['Wearing a mask is the worst way to prevent the virus , but using hand sanitizer is effective .', 'hand sanitizer']])
print(predictions)


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[0 0 0 1 0 1]
